In [1]:
import json
import requests
from google.cloud import bigquery
import logging
logging.basicConfig(level=logging.INFO)


sql = '''
select * from (
--注册打点
select 
       --某个包
       b.app_token,
       b.app_package_name,
       --某个用户
       b.backend,
       c.adid,
       b.user_id,
       --某个事件
       'register_account' as event_name,
       'jkcqdd' as event_token,--事件识别码
       --发生时间
       b.biz_time as event_time,--携带时区
       b.created_at as created_at_unix,--固定utc+0，ad调节时区，提供时间戳
       --产生收益
       cast(0 as float64) as amount,
       --打点时间/IP/设备型号
       FORMAT_DATETIME('%Y-%m-%d %H:%M:%S',current_datetime()) as cur_time,
       b.fst_log_ip,
       b.fst_log_device,
       
from `slots-tech.br_raw.user` b 
join `slots-tech.br_raw.ad_install_user` c on b.backend = c.backend and b.user_id = c.user_id 
left join `slots-tech.adjust.ad_add_event_history` h on b.backend = h.backend and c.adid = h.adid and c.user_id = h.user_id and b.biz_time = h.event_time and h.event_name = 'register_account' 
where h.adid is null --未打过点
and b.app_token is not null --能正常打点

--充值成功打点
union all
select 
       --某个包
       b.app_token,
       b.app_package_name,
       --某个用户
       b.backend,
       c.adid,
       a.user_id,
       --某个事件
       'recharge_success' as event_name,
       '4osdug' as event_token,--事件识别码
       --发生时间
       a.paid_time as event_time,--携带时区
       a.arrival_time as created_at_unix,--固定utc+0，ad调节时区，提供时间戳
       --产生收益
       cast(a.amount as float64) as amount,
       --打点时间/IP/设备型号
       FORMAT_DATETIME('%Y-%m-%d %H:%M:%S',current_datetime()) as cur_time,
       b.fst_log_ip,
       b.fst_log_device,

from `slots-tech.br_raw.b_recharge` a
join `slots-tech.br_raw.user` b on a.backend = b.backend and a.user_id = b.user_id
join `slots-tech.br_raw.ad_install_user` c on b.backend = c.backend and b.user_id = c.user_id 
left join `slots-tech.adjust.ad_add_event_history` h on c.backend = h.backend and c.adid = h.adid and c.user_id = h.user_id and a.paid_time = h.event_time and h.event_name = 'recharge_success'
where h.adid is null --未打过点
and b.app_token is not null --能正常打点
and a.paid_status = 1 --充值成功

--提现成功打点
union all
Select 
       --某个包
       b.app_token,
       b.app_package_name,
       --某个用户
       b.backend,
       c.adid,
       a.user_id,
       --某个事件
       'withdraw_success' as event_name,
       'cxstot' as event_token,--事件识别码
       --发生时间
       a.drawn_time as event_time,--携带时区
       a.arrival_time as created_at_unix,--固定utc+0，ad调节时区，提供时间戳
       --产生收益
       cast(a.amount as float64) as amount,
       --打点时间/IP/设备型号
       FORMAT_DATETIME('%Y-%m-%d %H:%M:%S',current_datetime()) as cur_time,
       b.fst_log_ip,
       b.fst_log_device,

from `slots-tech.br_raw.b_withdraw` a
join `slots-tech.br_raw.user` b on a.backend = b.backend and a.user_id = b.user_id
join `slots-tech.br_raw.ad_install_user` c on b.backend = c.backend and b.user_id = c.user_id 
left join `slots-tech.adjust.ad_add_event_history` h on c.backend = h.backend and c.adid = h.adid and c.user_id = h.user_id and a.drawn_time = h.event_time and h.event_name = 'withdraw_success'
where h.adid is null --未打过点
and b.app_token is not null --能正常打点
and a.drawn_status = 1 --提现成功

--新用户首次充值成功打点
union all
select a.app_token,a.app_package_name,a.backend,a.adid,a.user_id,a.event_name,a.event_token,a.event_time,a.created_at_unix,a.amount,a.cur_time,a.fst_log_ip,a.fst_log_device
from (
      select 
            *  
      from
      (
            select
                  --某个包
                  b.app_token,
                  b.app_package_name, 
                  --某个用户
                  b.backend,
                  c.adid,
                  a.user_id,
                  --某个事件
                 'recharge_success_newuser' as event_name,
                 'gv5loq' as event_token,--事件识别码
                  --发生时间
                  a.paid_time as event_time,--携带时区
                  a.arrival_time as created_at_unix,--固定utc+0，ad调节时区，提供时间戳
                  --产生收益
                  cast(a.amount as float64) as amount,
                  --打点时间/IP/设备型号
                  FORMAT_DATETIME('%Y-%m-%d %H:%M:%S',current_datetime()) as cur_time,
                  b.fst_log_ip,
                  b.fst_log_device,
                  row_number()over(partition by c.user_id order by a.paid_time) as rnk -- 一个ad_id可能对应多个user_id，要ad_id的首次数据，分组partition by adid
            from `slots-tech.br_raw.b_recharge` a
            join `slots-tech.br_raw.user` b on a.backend = b.backend and a.user_id = b.user_id
            join `slots-tech.br_raw.ad_install_user` c on b.backend = c.backend and b.user_id = c.user_id
            where a.paid_status = 1 and a.paid_day = b.biz_day -- 注册当天充值成功的数据
                  and b.app_token is not null --能正常打点
            )
      where rnk = 1 --取首次的充值
      ) a
left join `slots-tech.adjust.ad_add_event_history` h 
on a.backend = h.backend and a.adid = h.adid and a.user_id = h.user_id and a.event_time = h.event_time and h.event_name = 'recharge_success_newuser'
where h.adid is null --未打过点

--登录打点
union all
select 
       --某个包
       b.app_token,
       b.app_package_name,
       --某个用户
       b.backend,
       c.adid,
       a.user_id,
       --某个事件
       'login_success' as event_name,
       'l2pa7j' as event_token,--事件识别码
       --发生时间
       a.biz_time as event_time,--携带时区
       a.created_at as created_at_unix,--固定utc+0，ad调节时区，提供时间戳
       --产生收益
       cast(0 as float64) as amount,
       --打点时间/IP/设备型号
       FORMAT_DATETIME('%Y-%m-%d %H:%M:%S',current_datetime()) as cur_time,
       b.fst_log_ip,
       b.fst_log_device,

from `slots-tech.br_raw.b_user_logs` a
join `slots-tech.br_raw.user` b on a.backend = b.backend and a.user_id = b.user_id
join `slots-tech.br_raw.ad_install_user` c on b.backend = c.backend and b.user_id = c.user_id 
left join `slots-tech.adjust.ad_add_event_history` h on c.backend = h.backend and c.adid = h.adid and c.user_id = h.user_id and a.biz_time = h.event_time and h.event_name = 'login_success'
where h.adid is null --未打过点
and b.app_token is not null --能正常打点

--fb登录打点
union all
select 
       --某个包
       b.app_token,
       b.app_package_name,
       --某个用户
       b.backend,
       c.adid,
       a.user_id,
       --某个事件
       'register_account_fb' as event_name,--命名问题，本质是用fb登录事件
       '3vadcm' as event_token,--事件识别码
       --发生时间
       a.biz_time as event_time,--携带时区
       a.created_at as created_at_unix,--固定utc+0，ad调节时区，提供时间戳
       --产生收益
       cast(0 as float64) as amount,
       --打点时间
       FORMAT_DATETIME('%Y-%m-%d %H:%M:%S',current_datetime()) as cur_time,
       b.fst_log_ip,
       b.fst_log_device,

from `slots-tech.br_raw.b_user_logs` a
join `slots-tech.br_raw.user` b on a.backend = b.backend and a.user_id = b.user_id
join `slots-tech.br_raw.ad_install_user` c on b.backend = c.backend and b.user_id = c.user_id 
left join `slots-tech.adjust.ad_add_event_history` h on c.backend = h.backend and c.adid = h.adid and c.user_id = h.user_id and a.biz_time = h.event_time and h.event_name = 'register_account_fb'
where h.adid is null --未打过点
and b.app_token is not null --能正常打点
and login_type = 'facebook' --fb登录事件

--gg登录打点
union all
select 
       --某个包
       b.app_token,
       b.app_package_name,
       --某个用户
       b.backend,
       c.adid,
       a.user_id,
       --某个事件
       'register_account_gg' as event_name,--命名问题，本质是用gg登录事件
       'fmvpdr' as event_token,--事件识别码
       --发生时间
       a.biz_time as event_time,--携带时区
       a.created_at as created_at_unix,--固定utc+0，ad调节时区，提供时间戳
       --产生收益
       cast(0 as float64) as amount,
       --打点时间
       FORMAT_DATETIME('%Y-%m-%d %H:%M:%S',current_datetime()) as cur_time,
       b.fst_log_ip,
       b.fst_log_device,

from `slots-tech.br_raw.b_user_logs` a
join `slots-tech.br_raw.user` b on a.backend = b.backend and a.user_id = b.user_id
join `slots-tech.br_raw.ad_install_user` c on b.backend = c.backend and b.user_id = c.user_id 
left join `slots-tech.adjust.ad_add_event_history` h on c.backend = h.backend and c.adid = h.adid and c.user_id = h.user_id and a.biz_time = h.event_time and h.event_name = 'register_account_gg'
where h.adid is null --未打过点
and b.app_token is not null --能正常打点
and login_type = 'google' --gg登录事件

--phone登录打点
union all
select 
       --某个包
       b.app_token,
       b.app_package_name,
       --某个用户
       b.backend,
       c.adid,
       a.user_id,
       --某个事件
       'register_account_phone' as event_name,--命名问题，本质是用phone登录事件
       '50in6i' as event_token,--事件识别码
       --发生时间
       a.biz_time as event_time,--携带时区
       a.created_at as created_at_unix,--固定utc+0，ad调节时区，提供时间戳
       --产生收益
       cast(0 as float64) as amount,
       --打点时间
       FORMAT_DATETIME('%Y-%m-%d %H:%M:%S',current_datetime()) as cur_time,
       b.fst_log_ip,
       b.fst_log_device,

from `slots-tech.br_raw.b_user_logs` a
join `slots-tech.br_raw.user` b on a.backend = b.backend and a.user_id = b.user_id
join `slots-tech.br_raw.ad_install_user` c on b.backend = c.backend and b.user_id = c.user_id 
left join `slots-tech.adjust.ad_add_event_history` h on c.backend = h.backend and c.adid = h.adid and c.user_id = h.user_id and a.biz_time = h.event_time and h.event_name = 'register_account_phone'
where h.adid is null --未打过点
and b.app_token is not null --能正常打点
and login_type = 'phone' --phone登录事件

--guest登录打点
union all
select 
       --某个包
       b.app_token,
       b.app_package_name,
       --某个用户
       b.backend,
       c.adid,
       a.user_id,
       --某个事件
       'register_account_guest' as event_name,--命名问题，本质是用guest登录事件
       '6ag77n' as event_token,--事件识别码
       --发生时间
       a.biz_time as event_time,--携带时区
       a.created_at as created_at_unix,--固定utc+0，ad调节时区，提供时间戳
       --产生收益
       cast(0 as float64) as amount,
       --打点时间
       FORMAT_DATETIME('%Y-%m-%d %H:%M:%S',current_datetime()) as cur_time,
       b.fst_log_ip,
       b.fst_log_device,

from `slots-tech.br_raw.b_user_logs` a
join `slots-tech.br_raw.user` b on a.backend = b.backend and a.user_id = b.user_id
join `slots-tech.br_raw.ad_install_user` c on b.backend = c.backend and b.user_id = c.user_id 
left join `slots-tech.adjust.ad_add_event_history` h on c.backend = h.backend and c.adid = h.adid and c.user_id = h.user_id and a.biz_time = h.event_time and h.event_name = 'register_account_guest'
where h.adid is null --未打过点
and b.app_token is not null --能正常打点
and login_type = 'guest' --guest登录事件
)
where event_time >= '2023-11-20'--历史数据量太大，无法传输，舍弃
order by event_time --收入相关会受到时间序列检查，必须排序
;
'''

table_id = "slots-tech.adjust.ad_add_event_history"



In [2]:
bq_client = bigquery.Client.from_service_account_json(r"D:\1-script\3-PYTHON\1-游戏项目\5-bq钥匙访问\slots-tech.json")
df = bq_client.query(sql).to_dataframe()
print(df.head())
df['amount'] = df['amount'].astype('float64')
df['event_time'] = df['event_time'].astype('str')

      app_token    app_package_name backend                              adid  \
0  ldckwwrfzmkg  com.crazy.spin0627    br01  e3def6f967a8940a0726604c351bcedd   
1  ldckwwrfzmkg  com.crazy.spin0627    br01  e3def6f967a8940a0726604c351bcedd   
2  ldckwwrfzmkg  com.crazy.spin0627    br01  e3def6f967a8940a0726604c351bcedd   
3  ldckwwrfzmkg  com.crazy.spin0627    br01  e3def6f967a8940a0726604c351bcedd   
4  ldckwwrfzmkg  com.crazy.spin0627    br01  00377f6b712c702ca3f863c285bca9af   

  user_id              event_name event_token          event_time  \
0  104666           login_success      l2pa7j 2023-11-21 07:34:55   
1  104666           login_success      l2pa7j 2023-11-21 07:34:55   
2  104666  register_account_guest      6ag77n 2023-11-21 07:34:55   
3  104666  register_account_guest      6ag77n 2023-11-21 07:34:55   
4  113544           login_success      l2pa7j 2023-11-21 07:35:15   

   created_at_unix  amount             cur_time       fst_log_ip  \
0       1700562895     0.0  20

In [3]:

def add_event(event_name, created_at_unix, app_token, event_token, adid, amount, ip_address, user_agent):
    #这是控制是否是测试环境的，沙盒就是测试数据
    # environment = "sandbox"
    environment = "production"

    # The endpoint URL
    url = f"https://s2s.adjust.com/event"

    # The payload 非收入相关事项不要传递revenue和currency，会报错；另外打点需要user_agent和ip才能和facebook交互
    if event_name in ('recharge_success','withdraw_success','recharge_success_newuser'):

        payload = {
            "s2s":"1",
            "created_at_unix":created_at_unix, 
            "app_token": app_token,
            "event_token": event_token,
            "adid": adid,
            "revenue": amount,
            "currency": "BRL",
            "environment" :environment,
            "ip_address":ip_address,
            "user_agent":user_agent,
            # Add other parameters as needed
        }
    else:
        payload = {
            "s2s":"1",
            "created_at_unix":created_at_unix, 
            "app_token": app_token,
            "event_token": event_token,
            "adid": adid,
            # "revenue": amount,
            # "currency": "BRL",
            "environment" :environment,
            "ip_address":ip_address,
            "user_agent":user_agent,
            # Add other parameters as needed
        }

    # Send the POST request
    response = requests.post(url, data=payload)

    # Print the response
    if response.status_code == 200:
        logging.info("推送成功")
    else:
        logging.info(f"推送失败",response.text)
    return response.status_code == 200




In [4]:
t = 0
df.loc[t, ]

app_token                                                ldckwwrfzmkg
app_package_name                                   com.crazy.spin0627
backend                                                          br01
adid                                 e3def6f967a8940a0726604c351bcedd
user_id                                                        104666
event_name                                              login_success
event_token                                                    l2pa7j
event_time                                        2023-11-21 07:34:55
created_at_unix                                            1700562895
amount                                                            0.0
cur_time                                          2023-12-26 12:08:11
fst_log_ip                                            186.216.182.226
fst_log_device      Android OS 9 / API-28 (PPR1.180610.011/G611MTV...
Name: 0, dtype: object

In [6]:


success = add_event(df.loc[t,'event_name'], df.loc[t,'created_at_unix'], df.loc[t,'app_token'], df.loc[t,'event_token'], df.loc[t,'adid'], df.loc[t,'amount'], df.loc[t,'fst_log_ip'], df.loc[t,'fst_log_device'] )
if success:
    logging.info("开始插入add_event_history")
    rows_to_insert = [
                    {
                    "app_token":df.loc[i,'app_token'],
                    "app_package_name":df.loc[i,'app_package_name'],
                    "backend":df.loc[i,'backend'],
                    "adid":df.loc[i,'adid'],
                    "user_id":df.loc[i,'user_id'],
                    "event_name":df.loc[i,'event_name'],
                    "event_token":df.loc[i,'event_token'],
                    "event_time":df.loc[i,'event_time'],
                    "created_at_unix":str(df.loc[i,'created_at_unix']),
                    "amount":df.loc[i,'amount'],
                    "cur_time":df.loc[i,'cur_time'],
                    "fst_log_ip":df.loc[i,'fst_log_ip'],
                    "fst_log_device":df.loc[i,'fst_log_device'],
                    }
                ]
    errors = bq_client.insert_rows_json(table_id, rows_to_insert)


INFO:root:推送成功
INFO:root:开始插入add_event_history


NameError: name 'i' is not defined